# Deep Dive on Dask Tasks

In a perfect world or, more realistically, in an imperfect world where we aren't pushing our computational systems very hard (consider a consumer laptop end-user) we rarely or never run into thorny bugs or performance limits.

But whether we're crunching huge numbers of citizen tax records or hand-coding a massive-scale traffic simulation, the use cases that bring us to Dask are also likely -- sooner or later -- to require a bit more fine-tuning to get the best results.

So we're going to take a deep dive with the goal of knowing how we can get the best performance out of our system (and how to avoid common problems.)

In [ ]:
import coiled
from dask.distributed import Client

cluster = coiled.Cluster(name="training-cluster")
client = Client(cluster)
client

## Tasks

We've seen that, fundamentally, Dask tasks are just Python functions, and we've seen that we can run them via APIs like `client.submit`, `.compute`, and `.persist`

So long as our function's results are can be serialized via `cloudpickle`, we can get them back in our main process (the process where the `Client` instance lives).

But we should probably assume that we *don't* want to do that: round tripping objects over the network creates overhead, and touching them (deserializing, at least) in our main process can make our main code a bottleneck.

The general patterns should be:

__Only final results that are small and/or critical to the next step in our code__ should come back to our main process. These might be moderately-sized final reports, or intermediate values that we want to use for something else (e.g., parameter estimates from a model).

__Large final results__ should be written to stable storage (typically something like S3 or Kafka) in parallel, from the tasks that produce the output. Think about Dask dataframe's output methods, which write to shared-storage destinations in parallel/asynchronous tasks that product partition outputs: we should use those approaches or implement similar patterns on our own.

__Intermediate results which become inputs to subsequent tasks__ should stay "in the cluster"
* We use handles (Future or delayed results) to supply those outputs to subsequent tasks (function calls)
* Dask will automatically handle data locality optimization and data movement if necessary
    
Do this:

<img src='images/do.png'>

Not this:

<img src='images/dont.png'>

*It is totally understandable to want to get, touch, inspect, and debug intermediate results when you are developing code.*

Just make sure to streamline compositions for better deploy-time performance.

If your objects are big enough, you'll get warnings or even failures. But, otherwise, you may just get inefficient code.

__Task Granularity__

Dask offers a lot of flexibility around task granularity, but for every job and dataset, there will be some range that achieves optimal parallelism and utilization.

Dask's scheduler spends hundreds of microseconds per task; to minimize overhead ratio, your tasks should be orders of magnitude larger

__Task Scheduling__

The high-level "Journey of a task" below is taken from https://distributed.dask.org/en/latest/journey.html and is definitely worth reviewing together:

---

### Journey of a Task

We follow a single task through the user interface, scheduler, worker nodes, and back. Hopefully this helps to illustrate the inner workings of the system.

#### User code[](https://distributed.dask.org/en/latest/journey.html#user-code "Permalink to this headline")

A user computes the addition of two variables already on the cluster, then pulls the result back to the local process.

```python
client = Client('host:port')
x = client.submit(...)
y = client.submit(...)

z = client.submit(add, x, y)  # we follow z

print(z.result())
```

#### Step 1: Client[](https://distributed.dask.org/en/latest/journey.html#step-1-client "Permalink to this headline")

`z` begins its life when the `Client.submit` function sends the following message to the `Scheduler`:

```
{'op': 'update-graph',
 'tasks': {'z': (add, x, y)},
 'keys': ['z']}
```

The client then creates a `Future` object with the key `'z'` and returns that object back to the user. This happens even before the message has been received by the scheduler. The status of the future says `'pending'`.

#### Step 2: Arrive in the Scheduler[](https://distributed.dask.org/en/latest/journey.html#step-2-arrive-in-the-scheduler "Permalink to this headline")

A few milliseconds later, the scheduler receives this message on an open socket.

The scheduler updates its state with this little graph that shows how to compute `z`:

`scheduler.update_graph(tasks=msg['tasks'], keys=msg['keys'])`

The scheduler also updates *a lot* of other state. Notably, it has to identify that `x` and `y` are themselves variables, and connect all of those dependencies. This is a long and detail oriented process that involves updating roughly 10 sets and dictionaries. Interested readers should investigate `distributed/scheduler.py::update_graph()`. While this is fairly complex and tedious to describe rest assured that it all happens in constant time and in about a millisecond.

#### Step 3: Select a Worker[](https://distributed.dask.org/en/latest/journey.html#step-3-select-a-worker "Permalink to this headline")

Once the latter of `x` and `y` finishes, the scheduler notices that all of `z`'s dependencies are in memory and that `z` itself may now run. Which worker should `z` select? We consider a sequence of criteria:

1.  First, we quickly downselect to only those workers that have either `x` or `y` in local memory.
2.  Then, we select the worker that would have to gather the least number of bytes in order to get both `x` and `y` locally. E.g. if two different workers have `x` and `y` and if `y` takes up more bytes than `x` then we select the machine that holds `y` so that we don't have to communicate as much.
3.  If there are multiple workers that require the minimum number of communication bytes then we select the worker that is the least busy

`z` considers the workers and chooses one based on the above criteria. In the common case the choice is pretty obvious after step 1. `z` waits on a stack associated with the chosen worker. The worker may still be busy though, so `z` may wait a while.

#### Step 4: Transmit to the Worker[](https://distributed.dask.org/en/latest/journey.html#step-4-transmit-to-the-worker "Permalink to this headline")

Eventually the worker finishes a task, has a spare core, and `z` finds itself at the top of the stack (note, that this may be some time after the last section if other tasks placed themselves on top of the worker's stack in the meantime.)

We place `z` into a `worker_queue` associated with that worker and a `worker_core` coroutine pulls it out. `z`'s function, the keys associated to its arguments, and the locations of workers that hold those keys are packed up into a message that looks like this:

```
{'op': 'compute',
 'function': execute_task,
 'args': ((add, 'x', 'y'),),
 'who_has': {'x': {(worker_host, port)},
             'y': {(worker_host, port), (worker_host, port)}},
 'key': 'z'}
```

This message is serialized and sent across a TCP socket to the worker.

#### Step 5: Execute on the Worker[](https://distributed.dask.org/en/latest/journey.html#step-5-execute-on-the-worker "Permalink to this headline")

The worker unpacks the message, and notices that it needs to have both `x` and `y`. If the worker does not already have both of these then it gathers them from the workers listed in the `who_has` dictionary also in the message. For each key that it doesn't have it selects a valid worker from `who_has` at random and gathers data from it.

After this exchange, the worker has both the value for `x` and the value for `y`. So it launches the computation `add(x, y)` in a local `ThreadPoolExecutor` and waits on the result.

*In the mean time the worker repeats this process concurrently for other tasks. Nothing blocks.*

Eventually the computation completes. The Worker stores this result in its local memory:

```
data['z'] = ...
```

And transmits back a success, and the number of bytes of the result:

Worker: Hey Scheduler, 'z' worked great.
        I'm holding onto it.
        It takes up 64 bytes.

The worker does not transmit back the actual value for `z`.

#### Step 6: Scheduler Aftermath[](https://distributed.dask.org/en/latest/journey.html#step-6-scheduler-aftermath "Permalink to this headline")

The scheduler receives this message and does a few things:

1.  It notes that the worker has a free core, and sends up another task if available
2.  If `x` or `y` are no longer needed then it sends a message out to relevant workers to delete them from local memory.
3.  It sends a message to all of the clients that `z` is ready and so all client `Future` objects that are currently waiting should, wake up. In particular, this wakes up the `z.result()` command executed by the user originally.

#### Step 7: Gather[](https://distributed.dask.org/en/latest/journey.html#step-7-gather "Permalink to this headline")

When the user calls `z.result()` they wait both on the completion of the computation and for the computation to be sent back over the wire to the local process. Usually this isn't necessary, usually you don't want to move data back to the local process but instead want to keep in on the cluster.

But perhaps the user really wanted to actually know this value, so they called `z.result()`.

The scheduler checks who has `z` and sends them a message asking for the result. This message doesn't wait in a queue or for other jobs to complete, it starts instantly. The value gets serialized, sent over TCP, and then deserialized and returned to the user (passing through a queue or two on the way.)

#### Step 8: Garbage Collection[](https://distributed.dask.org/en/latest/journey.html#step-8-garbage-collection "Permalink to this headline")

The user leaves this part of their code and the local variable `z` goes out of scope. The Python garbage collector cleans it up. This triggers a decremented reference on the client (we didn't mention this, but when we created the `Future` we also started a reference count.) If this is the only instance of a Future pointing to `z` then we send a message up to the scheduler that it is OK to release `z`. The user no longer requires it to persist.

The scheduler receives this message and, if there are no computations that might depend on `z` in the immediate future, it removes elements of this key from local scheduler state and adds the key to a list of keys to be deleted periodically. Every 500 ms a message goes out to relevant workers telling them which keys they can delete from their local memory. The graph/recipe to create the result of `z` persists in the scheduler for all time.

#### Overhead[](https://distributed.dask.org/en/latest/journey.html#overhead "Permalink to this headline")

The user experiences this in about 10 milliseconds, depending on network latency.

--- 

Because Dask is highly flexible, the task scheduling heuristics are somewhat complex. You are unlikely to have to tinker with them, but they are, of course, customizable if you truly need it.

Lower-level details on how a worker is chosen for a task and how tasks are chosen to run (when many may be ready to run) are described at 
* https://distributed.dask.org/en/latest/scheduling-policies.html
* https://distributed.dask.org/en/latest/priority.html

Detailed state-machine logic for scheduling is at https://distributed.dask.org/en/latest/scheduling-state.html

### Your tasks ... in the cluster

You can programmatically locate your tasks in the cluster through `Client` APIs, but your first stop should probably be the dashboards, where it's easier to make high-level sense of what is going on.

In [ ]:
client.restart()

In [ ]:
def step1f():
    return [3.14159]*1000000

def step2f(x):
    return [2*i for i in x]

def step3f(x):
    return [i/3.0 for i in x]

af = client.submit(step1f)
bf = client.submit(step2f, af)
cf = client.submit(step3f, bf)

cf

In [ ]:
cf.status

Let's examine some of the dashboards and review what they tell us about these tasks. 

Dask has added a lot of new dashboards, which are all great -- but some may be more useful than others depending upon how your workloads stress your clusters.

So don't worry about try to monitor all the dashboards all the time (unless you have a lot of monitors in which case you can have a lot of fun making your office look like the USS Enterprise bridge or Galactica's CIC)

Here is a summary of what you can learn:

__Aggregate Time Per Action__
* total time spent in compute vs. serialization vs. data transfer

__Bandwith Types, Bandwidth Workers, Workers Network, Workers Network Timeseries__
* network transfer for I/O: acquiring needed data/objects, shuffle, passing results

__Cluster Map__
* available workers
* tasks and data in transit to/from workers and scheduler
* currently executing task for each worker

__Compute Time Per Key__
* aggregate actual time by task type

__CPU, Workers CPU Timeseries__
* CPU usage

__Graph, Groups__
* task function name
* completion and memory status
* dependencies

__Memory By Key__
* aggregate actual memory usage by task type (function)

__NProcessing__
* task count currently executing for each worker

__Progress__
* number of scheduled tasks for each type
* number completed for each type

__Task Stream__
* worker assignment
* duration
* data transfer durations

__Workers Memory Workers Memory Timeseries__
* memory consumption for each worker

__Workers Disk, Workers Disk Timeseries__
* disk read/write for each worker

>
> __Note:__ as more dashboards are developed, some widgets may not be exposed right away in the point-and-click dashboard tools. But you can find paths to them under `{dashboard server}/sitemap.json`
>

Dashboard data can also be programmatically collected

In [ ]:
client.get_task_stream()

and related methods like `get_profile` and `get_worker_logs`

as well written to files

In [ ]:
client.get_task_stream(filename='task_stream.html', plot='save')

#### Performance report context manager

We can export a larger set of reports to static HTML using Dask's `performance_report` context manager.

This context manager collects data for an arbitrary wrapped chunk of code, and generates a file.

Let's run an expensive task and generate a perf report.

In [ ]:
import numpy as np
from distributed import performance_report

def slow_sum(arr):
    total = 0
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            total = total + arr[i,j]
    return total
    
def sum_data():
    arr = np.random.normal(size=(3000,3000))
    return slow_sum(arr)

def do_long_work():
    return sum_data()

with performance_report(filename='report.html'):
    sumf = client.submit(do_long_work, pure=False)
    sumf.result()

__Programmatic or service access to metrics__

As with `client.get_task_stream()` it is often convenient to get performance data via code (receiving data structures that you don't have to parse) or via a web-service endpoint that can be monitored with scripts or standard monitoring tools like *Prometheus* (https://prometheus.io/)

Some additional metrics are exposed as data under
* https://docs.dask.org/en/latest/setup/prometheus.html
as well as under a few of the endpoints documented at
* https://distributed.dask.org/en/latest/http_services.html

`Client` APIs also offer some programmatic details on task placement:

In [ ]:
client.has_what()

In [ ]:
client.who_has(bf)

Probably all of the tasks from the recent sequence of operations are on one worker ... most likely because Dask is wisely following the data locality principle:

__It is usually cheaper to move a computation to data is than to move data for computation.__

Dask schedules tasks according to this principle.

* In the common case distributed runs tasks on workers that already hold dependent data. 
* If you have a task f(x) that requires some data x then that task will very likely be run on the worker that already holds x.
* If a task requires data split among multiple workers, then the scheduler chooses to run the task on the worker that requires the least data transfer to it.

If we really wanted to run a task on a different worker, we can do so by passing a worker or list of workers that are acceptable to `submit` 

In [ ]:
other_worker = ['tls://10.4.1.216:40881'] # replace this: paste in the URI of a worker that does *not* have step2f's result
other_worker

In [ ]:
demo = client.submit(step2f, [1,2,3], workers=other_worker)

In [ ]:
client.has_what()

If you're trying to manage where your tasks get scheduled, it's definitely worth a design discussion to address why the scheduler's defaults aren't meeting your needs.

If data transfer and locality are an issue, an alternative approach to try is to distribute your data and let Dask distribute your tasks.

We'll take a look at the various ways to supply data to your tasks in our next exploration.